### Manipulation of PeaskDB de novo-assisted database search results of Trocas 7 (high water, April 2019) incubation samples LC-MS/MS data using python.

Starting with:

    PeaksDB search results (.csv) of PTM-optimized database searches
    These were all searched with 15 ppm precursor tolerance and 0.5 ppm fragement ion tolerance
    Exported at <1.0% FDR
    
Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the TROCAS # (e.g., 101) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

In [1]:
cd /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/307_TROCAS7_Fusion_Apr2021_PEAKS_182/

/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/307_TROCAS7_Fusion_Apr2021_PEAKS_182


In [2]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [3]:
ls 

 307_TROCAS7_Fusion_Apr2021_PEAKS_182_peptide.csv
 307_TROCAS7_Fusion_Apr2021_PEAKS_182_protein.csv
 307_TROCAS7_Fusion_Apr2021_PEAKS_182_protein.fasta
'DB search psm.csv'
'de novo only peptides.csv'
 peptides.pep.xml
 protein-peptides.csv


In [4]:
# read the CSV into a dataframe using the pandas read_csv function
peaksdbdup306 = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/307_TROCAS7_Fusion_Apr2021_PEAKS_182/307_TROCAS7_Fusion_Apr2021_PEAKS_182_peptide.csv")

# remove redundant rows
peaksdb306 = pd.DataFrame.drop_duplicates(peaksdbdup306)

print(peaksdb306.columns)

columns = ['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area', 'Fraction', 'Scan', 'Source_File',
       '#Spec', '#Spec', 'Accession', 'PTM',
       'AScore']

peaksdb306.columns = columns

#remmove # spec and accession columns because they mess parsing up

del peaksdb306['#Spec']
del peaksdb306['Accession']
del peaksdb306['PTM']
del peaksdb306['AScore']

mean_length = peaksdb306['Length'].mean()
print(mean_length)

print("# redundant peaksdb peptides in combined dataframe", len(peaksdbdup306))
print("# nonredundant peaksdb peptides in combined dataframe", len(peaksdb306))

#look at the dataframe
peaksdb306.head()

Index(['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area Trocas7-307-Chav', 'Fraction', 'Scan', 'Source File', '#Spec',
       '#Spec Trocas7-307-Chav', 'Accession', 'PTM', 'AScore'],
      dtype='object')
11.191304347826087
# redundant peaksdb peptides in combined dataframe 115
# nonredundant peaksdb peptides in combined dataframe 115


,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,Source_File
0,SC(+57.02)AAAGTEC(+57.02)LISGWGNTK,85.78,1881.8350,18,2.1,941.9268,93.16,3370000.0,111,16381,20210411_Trocas7_667_Chav307_DDA_120min_1.raw
1,SSGSSYPSLLQC(+57.02)LK,69.75,1525.7446,14,2.0,763.8811,101.67,253000.0,111,18330,20210411_Trocas7_667_Chav307_DDA_120min_1.raw
2,SGGGGGGGLGSGGSIR,69.51,1231.5905,16,2.0,616.8037,39.50,204000.0,111,6725,20210411_Trocas7_667_Chav307_DDA_120min_1.raw
3,LGEHNIDVLEGNEQFINAAK,66.48,2210.0967,20,0.8,737.7067,95.79,18200.0,111,17016,20210411_Trocas7_667_Chav307_DDA_120min_1.raw
4,LGEHN(+.98)IDVLEGNEQFINAAK,52.83,2211.0808,20,3.8,738.0370,99.30,8700.0,111,17823,20210411_Trocas7_667_Chav307_DDA_120min_1.raw


In [5]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaksdb306['A'] = peaksdb306['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaksdb306['C'] = peaksdb306['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaksdb306['D'] = peaksdb306['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaksdb306['E'] = peaksdb306['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaksdb306['F'] = peaksdb306['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaksdb306['G'] = peaksdb306['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaksdb306['H'] = peaksdb306['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaksdb306 output, there will be no isoleucines (they're lumped in with leucines)
peaksdb306['I'] = peaksdb306['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaksdb306['K'] = peaksdb306['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaksdb306['L'] = peaksdb306['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaksdb306['M'] = peaksdb306['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaksdb306['N'] = peaksdb306['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaksdb306['P'] = peaksdb306['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaksdb306['Q'] = peaksdb306['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaksdb306['R'] = peaksdb306['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaksdb306['S'] = peaksdb306['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaksdb306['T'] = peaksdb306['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaksdb306['V'] = peaksdb306['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaksdb306['W'] = peaksdb306['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaksdb306['Y'] = peaksdb306['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaksdb306['c-carb'] = peaksdb306['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaksdb306['m-oxid'] = peaksdb306['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaksdb306['n-deam'] = peaksdb306['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a lamba function to enumerate the # of deamidated Q's in each peptide
peaksdb306['q-deam'] = peaksdb306['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# create a column with 'stripped' peptide sequences using strip
peaksdb306['stripped_peptide'] = peaksdb306['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaksdb306['stripped_length'] = peaksdb306['stripped_peptide'].apply(len)

peaksdb306['NAAF_num.'] = peaksdb306['Area'] / peaksdb306['stripped_length']

# total the number of modifications in sequence
peaksdb306['ptm-total'] = peaksdb306['c-carb'] + peaksdb306['m-oxid'] + peaksdb306['n-deam'] + peaksdb306['q-deam']

# turn all isoleucines into leucines
# this helps later in comparing Unipept peptides to PeaksDB and Comet ones
peaksdb306['stripped_IL']= peaksdb306['stripped_peptide'].str.replace('I','L')

# write modified dataframe to new txt file
peaksdb306.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/306_CV_T24_GF_PDB/306C_CV_T24_GF_PDB.csv")

# check out the results
peaksdb306.head()

,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,...,Y,c-carb,m-oxid,n-deam,q-deam,stripped_peptide,stripped_length,NAAF_num.,ptm-total,stripped_IL
0,SC(+57.02)AAAGTEC(+57.02)LISGWGNTK,85.78,1881.8350,18,2.1,941.9268,93.16,3370000.0,111,16381,...,0,2,0,0,0,SCLISGWGNTK,11,306363.636364,2,SCLLSGWGNTK
1,SSGSSYPSLLQC(+57.02)LK,69.75,1525.7446,14,2.0,763.8811,101.67,253000.0,111,18330,...,1,1,0,0,0,SSGSSYPSLLQCLK,14,18071.428571,1,SSGSSYPSLLQCLK
2,SGGGGGGGLGSGGSIR,69.51,1231.5905,16,2.0,616.8037,39.50,204000.0,111,6725,...,0,0,0,0,0,SGGGGGGGLGSGGSIR,16,12750.000000,0,SGGGGGGGLGSGGSLR
3,LGEHNIDVLEGNEQFINAAK,66.48,2210.0967,20,0.8,737.7067,95.79,18200.0,111,17016,...,0,0,0,0,0,LGEHNIDVLEGNEQFINAAK,20,910.000000,0,LGEHNLDVLEGNEQFLNAAK
4,LGEHN(+.98)IDVLEGNEQFINAAK,52.83,2211.0808,20,3.8,738.0370,99.30,8700.0,111,17823,...,0,0,0,1,0,LGEHNIDVLEGNEQFINAAK,20,435.000000,1,LGEHNLDVLEGNEQFLNAAK


In [6]:
# keep only stripped peptide I/L and NAAF
db_306 = peaksdb306[['stripped_IL', 'Area', 'NAAF_num.']]

db_306.set_index('stripped_IL')

# write modified dataframe to new txt file
db_306.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/306C_CV_T24_GF_PDB_ILnaafs.csv")

db_306.head()

,stripped_IL,Area,NAAF_num.
0,SCLLSGWGNTK,3370000.0,306363.636364
1,SSGSSYPSLLQCLK,253000.0,18071.428571
2,SGGGGGGGLGSGGSLR,204000.0,12750.000000
3,LGEHNLDVLEGNEQFLNAAK,18200.0,910.000000
4,LGEHNLDVLEGNEQFLNAAK,8700.0,435.000000


### Exporting txt files of stripped peptides at confidence cutoffs:

In [7]:
# keep only peptide column peptides <1% FDR (this is what we exported)
pep306moddup = peaksdb306[["Peptide"]]

# keep only the stripped peptide column <1% FDR
# this is what we'll use for UniPept input, etc
pep306dup = peaksdb306[["stripped_peptide"]]

# deduplicate both of these lists
pep306mod = pep306moddup.drop_duplicates()
pep306 = pep306dup.drop_duplicates()

# print out the #s of modified and stripped peptides, deduplicated and not

print("Total modified PeaksDB peptides in 306:", len(pep306moddup))
print("Deduplicated modified PeaksDB peptides in 306:", len(pep306mod))
print("Total stripped PeaksDB peptides in 306:", len(pep306dup))
print("Deduplicated stripped PeaksDB peptides in 306:", len(pep306))

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing
pep306.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/306_CV_T24_GF_PDB/306C_CV_T24_GF_PDB_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/306_CV_T24_GF_PDB/306C_CV_T24_GF_PDB_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/306_CV_T24_GF_PDB/306C_CV_T24_GF_PDB_stripped_peptides.fas

# look at the stripped peptides
pep306.head()

Total modified PeaksDB peptides in 306: 115
Deduplicated modified PeaksDB peptides in 306: 115
Total stripped PeaksDB peptides in 306: 115
Deduplicated stripped PeaksDB peptides in 306: 111


,stripped_peptide
0,SCLISGWGNTK
1,SSGSSYPSLLQCLK
2,SGGGGGGGLGSGGSIR
3,LGEHNIDVLEGNEQFINAAK
5,RHPYFYAPELLFFAK


## NAAF correction and exporting files with AA and PTM totals:

In [8]:
# made a new dataframe that contains the suMN of certain columns in the modified
# peptide dataframe above 

index = ['sample total']

data = {'A': peaksdb306['A'].sum(),
        'C': peaksdb306['C'].sum(),
        'D': peaksdb306['D'].sum(),
        'E': peaksdb306['E'].sum(),
        'F': peaksdb306['F'].sum(),
        'G': peaksdb306['G'].sum(),
        'H': peaksdb306['H'].sum(),
        'I': peaksdb306['I'].sum(),
        'K': peaksdb306['K'].sum(),
        'L': peaksdb306['L'].sum(),
        'M': peaksdb306['M'].sum(),
        'N': peaksdb306['N'].sum(),
        'P': peaksdb306['P'].sum(),
        'Q': peaksdb306['Q'].sum(),
        'R': peaksdb306['R'].sum(),
        'S': peaksdb306['S'].sum(),
        'T': peaksdb306['T'].sum(),
        'V': peaksdb306['V'].sum(),
        'W': peaksdb306['W'].sum(),
        'Y': peaksdb306['Y'].sum(),
        'c-carb': peaksdb306['c-carb'].sum(),
        'm-oxid': peaksdb306['m-oxid'].sum(),
        'n-deam': peaksdb306['n-deam'].sum(),
        'q-deam': peaksdb306['q-deam'].sum(),
        'Total area': peaksdb306['Area'].sum(),
        'Total length': peaksdb306['stripped_length'].sum()
       }

totalpeaksdb306 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                              'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                              'c-carb', 'm-oxid', 
                                              'n-deam', 'q-deam', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaksdb306['% C w/ carb'] = totalpeaksdb306['c-carb'] / totalpeaksdb306['C'] 

# calculate percentage of M's that are oxidized
totalpeaksdb306['% M w/ oxid'] = totalpeaksdb306['m-oxid'] / totalpeaksdb306['M'] 

# calculate percentage of N's that are deamidated
totalpeaksdb306['% N w/ deam'] = totalpeaksdb306['n-deam'] / totalpeaksdb306['N'] 

# calculate percentage of N's that are deamidated
totalpeaksdb306['% Q w/ deam'] = totalpeaksdb306['q-deam'] / totalpeaksdb306['Q'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaksdb306['NAAF denom.'] = totalpeaksdb306['Total area'] / totalpeaksdb306['Total length']

# write modified dataframe to new txt file
totalpeaksdb306.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/306_CV_T24_GF_PDB/306C_CV_T24_GF_PDB_totals.csv")

totalpeaksdb306.head()

,A,C,D,E,F,G,H,I,K,L,...,m-oxid,n-deam,q-deam,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF denom.
sample total,83,18,26,47,50,197,26,82,81,128,...,2,6,7,315426632.0,1262,1.0,0.222222,0.1,0.194444,249941.863708


In [9]:
# use the calculated NAAF factor (in totalpeaksdb306 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

NAAF20 = 249941.863708

# we'll use -10lgP > 20 as an approximate cutoff for the <1.0% FDR which is what we actually have
peaksdb306['NAAF factor'] = (peaksdb306['NAAF_num.'])/NAAF20

# make a dataframe that contains only what we need: sequences, AAs, PTMN
peaksdb306_NAAF = peaksdb306[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', \
                              'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                              'c-carb', 'm-oxid', 'n-deam', 'q-deam']].copy()

# multiply the NAAF20 factor by the AAs to normalize its abundance by peak area and peptide length

peaksdb306_NAAF['A-NAAF20'] = peaksdb306_NAAF['A'] * peaksdb306['NAAF factor']
peaksdb306_NAAF['C-NAAF20'] = peaksdb306_NAAF['C'] * peaksdb306['NAAF factor']
peaksdb306_NAAF['D-NAAF20'] = peaksdb306_NAAF['D'] * peaksdb306['NAAF factor']
peaksdb306_NAAF['E-NAAF20'] = peaksdb306_NAAF['E'] * peaksdb306['NAAF factor']
peaksdb306_NAAF['F-NAAF20'] = peaksdb306_NAAF['F'] * peaksdb306['NAAF factor']
peaksdb306_NAAF['G-NAAF20'] = peaksdb306_NAAF['G'] * peaksdb306['NAAF factor']
peaksdb306_NAAF['H-NAAF20'] = peaksdb306_NAAF['H'] * peaksdb306['NAAF factor']
peaksdb306_NAAF['I-NAAF20'] = peaksdb306_NAAF['I'] * peaksdb306['NAAF factor']
peaksdb306_NAAF['K-NAAF20'] = peaksdb306_NAAF['K'] * peaksdb306['NAAF factor']
peaksdb306_NAAF['L-NAAF20'] = peaksdb306_NAAF['L'] * peaksdb306['NAAF factor']
peaksdb306_NAAF['M-NAAF20'] = peaksdb306_NAAF['M'] * peaksdb306['NAAF factor']
peaksdb306_NAAF['N-NAAF20'] = peaksdb306_NAAF['N'] * peaksdb306['NAAF factor']
peaksdb306_NAAF['P-NAAF20'] = peaksdb306_NAAF['P'] * peaksdb306['NAAF factor']
peaksdb306_NAAF['Q-NAAF20'] = peaksdb306_NAAF['Q'] * peaksdb306['NAAF factor']
peaksdb306_NAAF['R-NAAF20'] = peaksdb306_NAAF['R'] * peaksdb306['NAAF factor']
peaksdb306_NAAF['S-NAAF20'] = peaksdb306_NAAF['S'] * peaksdb306['NAAF factor']
peaksdb306_NAAF['T-NAAF20'] = peaksdb306_NAAF['T'] * peaksdb306['NAAF factor']
peaksdb306_NAAF['V-NAAF20'] = peaksdb306_NAAF['V'] * peaksdb306['NAAF factor']
peaksdb306_NAAF['W-NAAF20'] = peaksdb306_NAAF['W'] * peaksdb306['NAAF factor']
peaksdb306_NAAF['Y-NAAF20'] = peaksdb306_NAAF['Y'] * peaksdb306['NAAF factor']

# multiply the NAAF20 factor by the PTMN normalize its abundance by peak area and peptide length

peaksdb306_NAAF['ccarb-NAAF20'] = peaksdb306_NAAF['c-carb'] * peaksdb306_NAAF['NAAF factor']
peaksdb306_NAAF['moxid-NAAF20'] = peaksdb306_NAAF['m-oxid'] * peaksdb306_NAAF['NAAF factor']
peaksdb306_NAAF['ndeam-NAAF20'] = peaksdb306_NAAF['n-deam'] * peaksdb306_NAAF['NAAF factor']
peaksdb306_NAAF['qdeam-NAAF20'] = peaksdb306_NAAF['q-deam'] * peaksdb306_NAAF['NAAF factor']


# write the dataframe to a new csv
peaksdb306_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/306_CV_T24_GF_PDB/306C_CV_T24_GF_PDB_naaf.csv")

peaksdb306_NAAF.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,I,...,R-NAAF20,S-NAAF20,T-NAAF20,V-NAAF20,W-NAAF20,Y-NAAF20,ccarb-NAAF20,moxid-NAAF20,ndeam-NAAF20,qdeam-NAAF20
0,SCLISGWGNTK,1.225740,3,2,0,1,0,3,0,1,...,0.000000,2.451479,2.451479,0.000000,1.22574,0.000000,2.451479,0.0,0.00000,0.0
1,SSGSSYPSLLQCLK,0.072303,0,1,0,0,0,1,0,0,...,0.000000,0.361513,0.000000,0.000000,0.00000,0.072303,0.072303,0.0,0.00000,0.0
2,SGGGGGGGLGSGGSIR,0.051012,0,0,0,0,0,10,0,1,...,0.051012,0.153036,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.00000,0.0
3,LGEHNIDVLEGNEQFINAAK,0.003641,2,0,1,3,1,2,1,2,...,0.000000,0.000000,0.000000,0.003641,0.00000,0.000000,0.000000,0.0,0.00000,0.0
4,LGEHNIDVLEGNEQFINAAK,0.001740,2,0,1,3,1,2,1,2,...,0.000000,0.000000,0.000000,0.001740,0.00000,0.000000,0.000000,0.0,0.00174,0.0


In [10]:
### made a new dataframe that contains the suMN of NAAF normalized AAs for peaksdb306 peaksdb results
# also contains the suMN of the NAAF-corrected PTMN occurances for each affected residue

index = ['sample total']

data = {'NAAF': peaksdb306_NAAF['NAAF factor'].sum(),
        'A': peaksdb306_NAAF['A-NAAF20'].sum(),
        'C': peaksdb306_NAAF['C-NAAF20'].sum(),
        'D': peaksdb306_NAAF['D-NAAF20'].sum(),
        'E': peaksdb306_NAAF['E-NAAF20'].sum(),
        'F': peaksdb306_NAAF['F-NAAF20'].sum(),
        'G': peaksdb306_NAAF['G-NAAF20'].sum(),
        'H': peaksdb306_NAAF['H-NAAF20'].sum(),
        'I': peaksdb306_NAAF['I-NAAF20'].sum(),
        'K': peaksdb306_NAAF['K-NAAF20'].sum(),
        'L': peaksdb306_NAAF['L-NAAF20'].sum(),
        'M': peaksdb306_NAAF['M-NAAF20'].sum(),
        'N': peaksdb306_NAAF['N-NAAF20'].sum(),
        'P': peaksdb306_NAAF['P-NAAF20'].sum(),
        'Q': peaksdb306_NAAF['Q-NAAF20'].sum(),
        'R': peaksdb306_NAAF['R-NAAF20'].sum(),
        'S': peaksdb306_NAAF['S-NAAF20'].sum(),
        'T': peaksdb306_NAAF['T-NAAF20'].sum(),
        'V': peaksdb306_NAAF['V-NAAF20'].sum(),
        'W': peaksdb306_NAAF['W-NAAF20'].sum(),
        'Y': peaksdb306_NAAF['Y-NAAF20'].sum(),
        'c-carb': peaksdb306_NAAF['ccarb-NAAF20'].sum(),
        'm-oxid': peaksdb306_NAAF['moxid-NAAF20'].sum(),
        'n-deam': peaksdb306_NAAF['ndeam-NAAF20'].sum(),
        'q-deam': peaksdb306_NAAF['qdeam-NAAF20'].sum()
       }

totalpeaksdb306_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', \
                                                   'G', 'H', 'I','K', 'L', 'M', \
                                                   'N', 'P', 'Q', 'R', 'S', \
                                                   'T', 'V', 'W', 'Y', 'c-carb', \
                                                   'm-oxid', 'n-deam', 'q-deam'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, and Rs


# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaksdb306_NAAF['% C w/ carb.'] = totalpeaksdb306_NAAF['c-carb'] / totalpeaksdb306_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaksdb306_NAAF['% M w/ oxid'] = totalpeaksdb306_NAAF['m-oxid'] / totalpeaksdb306_NAAF['M'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb306_NAAF['% N w/ deam'] = totalpeaksdb306_NAAF['n-deam'] / totalpeaksdb306_NAAF['N'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb306_NAAF['% Q w/ deam'] = totalpeaksdb306_NAAF['q-deam'] / totalpeaksdb306_NAAF['Q'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaksdb306_NAAF['NAAF check'] = totalpeaksdb306_NAAF['NAAF'] / 249941.863708

# write modified dataframe to new txt file, same name + totals
totalpeaksdb306_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/306_CV_T24_GF_PDB/306C_CV_T24_GF_PDB_naaf_totals.csv")

totalpeaksdb306_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,Y,c-carb,m-oxid,n-deam,q-deam,% C w/ carb.,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF check
sample total,149.140463,102.339131,3.097946,2.864871,3.32907,3.3869,12.864394,5.001527,5.116223,5.625788,...,1.433187,3.097946,0.004965,1.919041,0.269489,1.0,0.001832,0.068913,0.053433,0.000597
